# Quick intro to Sympy

Sympy is the symbolic python library. Basically, it provides all the functionality of Maple in a python environment. Its compatibility with Numpy makes it very attractive for simplifying some calculations.

In [ ]:
import sympy as sp
sp.init_printing()  # generates nice formulas as output

The key to working with Sympy is defining symbols as an object. These can be used to construct mathematical expressions. For example, to define a symbol $x$:

In [ ]:
x = sp.Symbol('x')
x**2

Or multiple symbols at once:

In [ ]:
a,b,n = sp.symbols('a,b,n')
f = (a + x)**2 + b*x + 5
f

The function solve can be used to find the solution of (nonlinear) equations.

In [ ]:
sp.solve(sp.Eq(f,0), x)  # solve for f = 0 

Another useful function is simplify.

In [ ]:
g = sp.sin(x)**2 + sp.cos(x)**2 
g

In [ ]:
g.simplify()

Taking derivatives and integration is quite simple as well.

In [ ]:
h = (x**4 + 15*x**3 - 17*x)/(x**3-3)**3
h

In [ ]:
h.diff(x)

Note that you can turn sympy expression in python functions:

In [ ]:
dh_dx = sp.lambdify(x, h.diff(x))
dh_dx(2.2)

In [ ]:
(sp.log(x)).integrate(x)

or

In [ ]:
(1/x).integrate((x, 1, 2))  # integrate from 1 to 2

In [ ]:
from IPython.html.widgets import interact
from IPython.display import display

@interact
def factorize(n=(1, 20, 1)):
    expr = x**n - 1
    display(sp.Eq(expr, sp.factor(expr)))

We can also symbolically solve certain differntial equations

In [ ]:
N, t, r, K, N0 = sp.symbols('N, t, r, K, N_0')

Ndot = r * N(t) * (1 - N(t)/K)
logistic = sp.Eq(N(t).diff(t), Ndot)
logistic

In [ ]:
exact_solution = sp.dsolve(sp.powsimp(logistic), N(t))
exact_solution

For numerical methods, Sympy can be of use for calculating the Hessian, for example

In [ ]:
hessian = sp.Matrix(2, 2, [Ndot.diff(var1).diff(var2) for var1 in [K, r] for var2 in [K, r]])
hessian

Luckily, we can convert this into LaTeX code to imbed in our thesis, papers or course notes ;-)

In [ ]:
sp.latex(hessian)